In [1]:
import pymorphy2
import nltk
import ujson as json
import matplotlib.pyplot as plt
import numpy as np
import itertools
import gzip
from nltk.stem.snowball import RussianStemmer
from datetime import datetime
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
from gensim.models import ldamulticore
from gensim.models import LdaModel
from scipy.spatial.distance import cosine
from gensim.models import word2vec

from collections import Counter,OrderedDict

In [2]:
class Document:
    def __init__(self, init_dict):
        self.title = init_dict.get('title', '')
        self.description = init_dict.get('description', '')
        self.url = init_dict.get('url', '')
        self.site = init_dict.get('site', '')
        self.ts = datetime.fromtimestamp(init_dict['ts']) if 'ts' in init_dict else -1
    
    def __str__(self):
        res = ''
        res += 'url : %s\n' % self.url
        res += 'date : %s\n' % self.ts
        res += 'title : %s\n' % self.title
        res += 'description : %s\n' % self.description
        res += 'site : %s\n' % self.site
        return res

In [3]:
fin = gzip.open('dataset_mai.jsonl.gz')
for line in itertools.islice(fin, 10):
    data = json.loads(line.strip())
    print(Document(data))

url : http://bloknot-volzhsky.ru/news/volzhane-mogut-podat-zayavlenie-na-letnie-putevki-
date : 2019-11-30 18:26:10
title : Волжане могут подать заявление на летние путевки для детей
description : С понедельника заявления начинают принимать в МФЦ
site : bloknot-volzhsky.ru

url : https://trikky.ru/test-na-znanie-russkogo-yazyka-423354.html
date : 2019-11-30 18:26:48
title : 💗Тест на знание русского языка💗
description : Тест со сложными и легкими вопросами. Для кого-то будет легко набрать все 100 баллов, а кому-то будет немного тяжело. В любом случае попробовать стоит.1. Что изучает фразеология? способы образования слов устойчивые сочетания слов части речи2. На месте каких цифр в словах пишется н? В простенке между занавеше(1)ыми окнами были установле(2)ы часы, а рядом с ними […]
site : trikky.ru

url : https://topwar.ru/165315-chernomorskij-flot-popolnilsja-150-tonnym-plavkranom-proekta-02690.html
date : 2019-11-30 18:26:43
title : Черноморский флот пополнился 150-тонным плавкраном про

In [4]:
fin = gzip.open('dataset_mai.jsonl.gz')
dataset = []
dataset_test = []
for line in itertools.islice(fin, 10000):
    data = json.loads(line.strip())
    dataset.append(Document(data))

In [5]:
morth_analyzer = pymorphy2.MorphAnalyzer()

In [6]:
def split_words_v3(a_text):
    cur_word = ''
    prev_is_alpha = False

    for letter in a_text:
        if  (letter.isalpha() and prev_is_alpha or 
            letter.isdigit() and not prev_is_alpha):
            cur_word += letter
        elif (letter.isalpha() and not prev_is_alpha or
             letter.isdigit() and prev_is_alpha):
            if cur_word: yield cur_word
            cur_word = letter
            prev_is_alpha = not prev_is_alpha
        else:
            if cur_word: yield cur_word
            cur_word = ''
            prev_is_alpha = False
    if cur_word: yield cur_word
         


In [7]:
def get_norm_word_v3(a_word):
    MORTH_CACHE = {}
    if a_word not in MORTH_CACHE: MORTH_CACHE[a_word] = morth_analyzer.parse(a_word)[0].normal_form
    return MORTH_CACHE[a_word]

In [8]:
def get_doc_words(a_doc, a_split=split_words_v3, a_norm_word=get_norm_word_v3):
    for word in itertools.chain(a_split(a_doc.title), a_split(a_doc.description)):
        yield a_norm_word(word)

In [9]:
word2id = {}
for item in dataset:
    item.words = list(get_doc_words(item))
    for word in item.words:
        if word not in word2id: word2id[word] = len(word2id)

In [10]:
def get_vec_doc(a_doc,a_common_dictionary):
    indexs_word = a_common_dictionary.doc2idx(a_doc.words)
    result_array = np.zeros(len(a_common_dictionary))
    for i in indexs_word:
        result_array[i] += 1
    return result_array

def get_vec_w2v(a_doc,a_w2v):
    res_vec = np.zeros(len(a_w2v['bmw']))
    for word in a_doc.words:
        if word in a_w2v:
            res_vec =res_vec+a_w2v[word].copy()
    return res_vec/len(a_doc.words)

def interprecent_count(l_doc,r_doc):
    count  = 0
    res = []
    for word  in l_doc.values():
        if word in r_doc.values():
            res.append(word)
            count+=1
    return count - abs(len(l_doc) - len(r_doc))

## ДЗ - реализовать поиск похожих документов по текстовым векторам и по word2vec векторам


In [11]:
import operator
def get_word_match_most_similar_docs(a_doc, a_dataset, a_top_n=10):
    print('get word match most similar docs')
    a_doc_dict= Dictionary([a_doc.words])
    a_doc_index = a_dataset.index(a_doc)
    res = {}
    print(f'original: \n{a_doc}')
    for i, doc in enumerate(a_dataset):
        if i == a_doc_index:
            continue
        doc_dict = Dictionary([doc.words])
        res[i] = interprecent_count(a_doc_dict,doc_dict)    
    res =  sorted(res.items(), key=operator.itemgetter(1),reverse=True)
    print('result doc:')
    for index, c in res[:a_top_n]:
        print(dataset[index])
        
def get_tf_idf_most_similar_doc(a_doc, a_dataset, a_top_n=10):
    # для каждого документа строится вектор размерности словаря (аналогично random forest) и добавляется idf
    # далее cosine между документами
    print('get tf idf most similar doc')
    print(f'original: \n{a_doc}')
    common_dictionary = Dictionary(item.words for item in a_dataset)
    a_doc_index = a_dataset.index(a_doc)
    a_doc_vec = get_vec_doc(a_doc=a_doc,a_common_dictionary=common_dictionary)
    cosin_res = {}    
    for i, doc in enumerate(a_dataset):
        if i == a_doc_index:
            continue
        doc_vec_sim = get_vec_doc(doc,common_dictionary)
        cos_prom = cosine(a_doc_vec,doc_vec_sim)
        cosin_res[i] = cos_prom
    cosin_res = sorted(cosin_res.items(), key=operator.itemgetter(1))
    print('result:')
    for index, c in cosin_res[:a_top_n]:
        print(dataset[index])

        
def get_w2v_most_similar_doc(a_doc, a_dataset, a_top_n=10):
    # считается средний вектор по всем словам (можно при усреднении учитывать idf)
    print('get w2v most similar doc')
    print(f'original: \n{a_doc}')
    w2v = word2vec.Word2Vec([item.words for item in a_dataset], workers=4)
    a_doc_vec = get_vec_w2v(a_doc,w2v)
    a_doc_index = a_dataset.index(a_doc)
    cosin_res = {}
    for i, doc in enumerate(a_dataset):
        if i == a_doc_index:
            continue 
        doc_vec_sim = get_vec_w2v(doc,w2v)
        cos_prom = cosine(a_doc_vec,doc_vec_sim)
        cosin_res[i] = cos_prom
    cosin_res = sorted(cosin_res.items(), key=operator.itemgetter(1))
    print('result:')
    for index, c in cosin_res[:a_top_n]:
        print(dataset[index])

## Пример формата выдачи

#### Тестовые данные

In [12]:
doc_id = 13 #dota2
doc_id = 1946 #гороскоп
doc_id = 3388 #хоккей
doc_id = 7601 #телефоны

In [13]:
doc_id = 13 
for similart_func in [get_word_match_most_similar_docs,  get_tf_idf_most_similar_doc, get_w2v_most_similar_doc]:
    similart_func(dataset[doc_id], dataset)

get word match most similar docs
original: 
url : https://cyber.sports.ru/dota2/1080755627.html
date : 2019-11-30 18:26:39
title : Результаты Parimatch League Dota 2. Virtus.pro победила
description : 30 ноября завершился турнир Parimatch League. В финале Virtus.pro разгромила HellRaisers со счетом 3:0 и заработала 40 тысяч долларов. Лан-финал Parimatch League прошел с 28 по 30 ноября в Москве. 4 команды разыграли 70 тысяч долларов. Результаты команд 1. Virtus.pro2.
site : cyber.sports.ru

result doc:
url : https://cyber.sports.ru/dota2/1080757051.html
date : 2019-11-30 18:37:09
title : Призовой фонд Parimatch League Dota 2
description : 30 ноября в Москве завершился турнир Parimatch League. Призовой фонд турнира – 70 тысяч долларов. В финале Virtus.pro разгромила HellRaisers со счетом 3:0 и заработала 40 тысяч долларов. Распределение призовых 1. Virtus.pro – $40 0002. HellRaisers – $20 0003.
site : cyber.sports.ru

url : https://cyber.sports.ru/dota2/1080754953.html
date : 2019-11-30 

/home/hvidsmen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':
/home/hvidsmen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  # This is added back by InteractiveShellApp.init_path()
/home/hvidsmen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':
/home/hvidsmen/anaconda3/lib/python3.7/site-packages/scipy/spatial/distance.py:702: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


result:
url : https://cyber.sports.ru/dota2/1080757051.html
date : 2019-11-30 18:37:09
title : Призовой фонд Parimatch League Dota 2
description : 30 ноября в Москве завершился турнир Parimatch League. Призовой фонд турнира – 70 тысяч долларов. В финале Virtus.pro разгромила HellRaisers со счетом 3:0 и заработала 40 тысяч долларов. Распределение призовых 1. Virtus.pro – $40 0002. HellRaisers – $20 0003.
site : cyber.sports.ru

url : https://cyber.sports.ru/dota2/1080752587.html
date : 2019-11-30 16:57:20
title : Virtus.pro ведет со счетом 2:0 по картам в финале лиги Parimatch
description : Virtus.pro разгромила HellRaisers на второй карте гранд-финала Parimatch League. Матч закончился со счетом 29:23. Virtus.pro ведет со счетом 2-0. Серия проходит в формате bo5. Победитель получит 40 тысяч долларов, проигравшая команда – 20 тысяч долларов. Лан-финал Parimatch League проходит с 28 по 30 ноября в Москве.
site : cyber.sports.ru

url : https://cyber.sports.ru/dota2/1080754953.html
date : 2

In [14]:
doc_id = 1946 
for similart_func in [get_word_match_most_similar_docs,  get_tf_idf_most_similar_doc, get_w2v_most_similar_doc]:
    similart_func(dataset[doc_id], dataset)

get word match most similar docs
original: 
url : https://www.obozrevatel.com/astro/news/goroskop-na-1-dekabrya-chto-zhdet-lvov-rakov-dev-i-drugie-znaki-zodiaka.htm
date : 2019-11-30 17:07:11
title : Гороскоп на 1 декабря: что ждет Львов, Раков, Дев и другие знаки зодиака
description : Советы астрологов на воскресенье
site : obozrevatel.com

result doc:
url : https://prochepetsk.ru/news/19614
date : 2019-11-30 15:02:54
title : Романтичный ужин у Раков и шопинг у Водолеев: гороскоп для всех знаков зодиака
description : Гороскоп на 30 ноября и 1 декабря для всех знаков зодиака
site : prochepetsk.ru

url : https://www.gs.by/2019/11/30/goroskop-na-2020-god-beloj-krysy-ovnam-i-rakam-nuzhno-byt-nacheku/
date : 2019-11-30 16:41:48
title : Гороскоп на 2020 год Белой Крысы: Овнам и Ракам нужно быть начеку
description : Чего ожидать знакам Зодиака в приближающемся году
site : gs.by

url : https://progorod43.ru/news/68371
date : 2019-11-30 15:01:49
title : Близнецам потребуются силы, а Скорпионы 

/home/hvidsmen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':
/home/hvidsmen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  # This is added back by InteractiveShellApp.init_path()
/home/hvidsmen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


result:
url : https://www.obozrevatel.com/show/plemennoj-zherebets-gena-viter-otkrovenno-udivil-poklonnits-na-krasnoj-dorozhke-premii-m1.htm
date : 2019-11-30 18:02:08
title : Племенной жеребец: Гена Витер откровенно удивил поклонниц на красной дорожке премии М1
description : Gena VITER решил привлечь внимание
site : obozrevatel.com

url : https://megaobzor.com/Xiaomi-Mi-Mix-3-5G-i-ego-funkcii.html
date : 2019-11-30 19:26:42
title : Xiaomi Mi Mix 3 5G и его функции
description : Компания Xiaomi выпустила флагманскую новинку под названием Xiaomi Mi Mix 3 5G. В характеристики смартфона входит оперативная память на 6 Гбайт, флэш накопитель на 64 Гбайт. Также новинка имеет экран с диагональю 6,39 дюйма обладает разрешением 2340 × 1080 точек. Сердцем смартфона является процессор Qualcomm Snapdragon 855. О стоимости нового гаджета пока неизвестно.
site : megaobzor.com

url : https://citydog.by/post/zaden-prilozhenie-bilety/
date : 2019-11-30 15:06:45
title : Со 2 декабря появится новое прило

In [15]:
doc_id = 3388 
for similart_func in [get_word_match_most_similar_docs,  get_tf_idf_most_similar_doc, get_w2v_most_similar_doc]:
    similart_func(dataset[doc_id], dataset)

get word match most similar docs
original: 
url : https://www.tv21.ru/news/2019/11/30/khokkeisty-murmana-proveli-pervyy-domashniy-match-na-stadione-stroitel
date : 2019-11-30 15:31:47
title : Хоккеисты "Мурмана" провели первый домашний матч на стадионе "Строитель"
description : Игра состоялась в рамках открытия нового сезона Суперлиги по хоккею с мячом.
site : tv21.ru

result doc:
url : https://severpost.ru/read/87561
date : 2019-11-30 17:32:40
title : Первый домашний матч «Мурман» сыграл вничью
description : В своем первом домашнем матче «Мурман» не смог одолеть «Волгу». Игра проходила на стадионе «Строитель». В первом тайме мурманчане уступили хоккеистам из...
site : severpost.ru

url : https://sportmail.ru/news/hockey-bandy/39688181/
date : 2019-11-30 15:41:51
title : «Енисей» обыграл «Родину» в матче чемпионата России по хоккею с мячом
description : МОСКВА, 30 ноя — РИА Новости. Красноярский «Енисей» на выезде переиграл кировскую «Родину» в матче чемпионата России по хоккею с мячом

/home/hvidsmen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':
/home/hvidsmen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  # This is added back by InteractiveShellApp.init_path()
/home/hvidsmen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


result:
url : https://lenta.ru/news/2019/11/30/citymanchester/
date : 2019-11-30 17:37:16
title : Гол мощным ударом из-за штрафной помог «Манчестер Сити» добыть ничью в игре АПЛ
description : «Манчестер Сити» на выезде сыграл вничью с «Ньюкаслом» в матче 14-го тура Английской премьер-лиги (АПЛ). Встреча прошла в субботу, 30 ноября, и завершилась со счетом 2:2. В составе «горожан» в первом тайме отличился Рахим Стерлинг. Еще один мяч мощным ударом из-за пределов штрафной забил Кевин де Брейне.
site : lenta.ru

url : http://terrikon.com/posts/341367
date : 2019-11-30 19:07:42
title : 15-й тур чемпионата Франции: смотреть онлайн-видеотрансляции матчей
description : Игровой день во Франции начнется в 18:30. Именно тогда на поле выйдут "Страсбур" и "Лион". А в 21:00 пройдут сразу пять матчей. Речь идет о поединках "Лилль" - "Дижон", "Монпелье" - "Амьен", "Ним" - "Мец", "Ницца" - "Анже" и "Реймс" - "Бордо".
site : terrikon.com

url : https://www.euro-football.ru/article/35/1004281748_real_na

In [16]:
doc_id = 7601
for similart_func in [get_word_match_most_similar_docs,  get_tf_idf_most_similar_doc, get_w2v_most_similar_doc]:
    similart_func(dataset[doc_id], dataset)

get word match most similar docs
original: 
url : https://megaobzor.com/Stala-izvestna-cena-smartfona-Redmi-K30.html
date : 2019-11-30 12:12:16
title : Стала известна цена смартфона Redmi K30
description : Авторитетный искатель утечек Мукул Шарма поделился подробностями о цене смартфона Redmi K30, официальный анонс которого состоится уже 10 декабря. Если верить источнику, аппарат обойдется в 327 долларов, что намного больше 285 долларов, которые ему приписывали ранее. По предварительным данным, Redmi K30 получит аккумулятор ёмкостью 5000 мАч, квадрокамеру с главным датчиком изображения разрешением 60 Мп, дисплей с частотой обновления 120 Гц, двойную фронтальную камеру на 20 Мп и 2 Мп и восьмиядерный процессор Snapdragon 730G.
site : megaobzor.com

result doc:
url : https://megaobzor.com/Smartfon-LG-G8-ThinQ-poluchil-Android-10.html
date : 2019-11-30 17:47:14
title : Смартфон LG G8 ThinQ получил Android 10
description : Компания LG начала обновлять свой флагманский смартфон G8 ThinQ до 

result:
url : https://megaobzor.com/Pervii-kachestvennii-render-smartfona-Redmi-K30.html
date : 2019-11-30 10:34:36
title : Первый качественный рендер смартфона Redmi K30
description : Сегодня в сеть была опубликована первая официальная фотография смартфона Xiaomi Redmi K30, который анонсируют 10 декабря. Ею поделился авторитетный инсайдер под ником Xiaomishka. Как мы можем заметить, слухи про экран с одинаковыми рамками по бокам и над дисплеем, заметный подбородок и врезанной в него двойной фронтальной камерой оправдались. По остальным данным смартфон должен оснаститься 6,66- дюймовым LCD-дисплеем с разрешением Full HD+, чипсетом Snapdragon 730G, 6 ГБ оперативной памяти, 64 встроенной памяти, квадрокамерой на 60, 8, 13 и 2 Мп, встроенной в экран двойной фронтальной камерой на 20 и 2 Мп, NFC датчиком, дактилоскопом на торце, батареей на 5000 мАч, поддержкой 27- Вт быстрой зарядки и MIUI 11 на Android 10. Н
site : megaobzor.com

url : https://www.ixbt.com/news/2019/11/30/ubijstvennyj-re

/home/hvidsmen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':
/home/hvidsmen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  # This is added back by InteractiveShellApp.init_path()
/home/hvidsmen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


result:
url : https://megaobzor.com/Novaja-tehnologija-Honor-V30-Pro.html
date : 2019-11-30 18:26:15
title : Новая технология Honor V30 Pro
description : Компания Honor выпустила интересную новинку с отличной функцией под названием V30 Pro. Аппарат включает в себя дисплей на 6,57 дюйма,флэш накопитель на 256 Гбайт.Также имеется объем оперативной памяти 8 Гбайт. Сердцем смартфона является фирменный процессор Kirin 990. О стоимости нового гаджета пока неизвестно.
site : megaobzor.com

url : https://rozetked.me/news/8719-marketingovye-izobrazheniya-redmi-k30-s-dvoynoy-selfi-kameroy
date : 2019-11-30 17:03:37
title : Маркетинговые изображения Redmi K30 с двойной селфи-камерой
description : В социальной сети Weibo фигурируют пресс-изображения смартфона серии Redmi K30. Презентация новинок пройдёт в Китае 10 декабря. C фронтальной стороны отличительным элементом является двойная камера, размещённая в правом верхнем углу дисплея. С трёх сторон экрана рамки равны по толщине. Сзади заметен моду